In [1]:
import { initializeWasm } from '@docknetwork/crypto-wasm-ts'
await initializeWasm();

In [2]:
// Generating a keypair once signature parameters are created.
import { BBSKeypair, BBSSignatureParams, BBS_SIGNATURE_PARAMS_LABEL_BYTES } from '@docknetwork/crypto-wasm-ts';

const stringToBytes = (str: string) => Uint8Array.from(Buffer.from(str, "utf-8"));

const params = BBSSignatureParams.generate(100, BBS_SIGNATURE_PARAMS_LABEL_BYTES);
const keypair = BBSKeypair.generate(params, stringToBytes('seed1'));
const secretKey = keypair.secretKey;
const publicKey = keypair.publicKey;

In [4]:
import { BoundCheckSnarkSetup, SetupParam } from '@docknetwork/crypto-wasm-ts';
const provingKey = BoundCheckSnarkSetup();
const snarkProvingKey = provingKey.decompress();
const snarkVerifyingKey = provingKey.getVerifyingKeyUncompressed();

1


In [3]:
import { CredentialSchema, BBSCredentialBuilder, BBSCredential, SUBJECT_STR } from '@docknetwork/crypto-wasm-ts'

const schema1 = CredentialSchema.essential();
schema1.properties[SUBJECT_STR] = {
    type: 'object',
    properties: {
        fname: { type: 'string' },
        lname: { type: 'string' },
        BMI: { type: 'number', minimum: 0, multipleOf: 0.01 },
    }
}

const builder1 = new BBSCredentialBuilder();
builder1.schema = new CredentialSchema(schema1);
builder1.subject = {
    fname: 'John',
    lname: 'Smith',
    BMI: 23.25,
};

const credential = builder1.sign(secretKey);
credential.verify(publicKey)

{ verified: true, error: undefined }


In [5]:
import { PresentationBuilder } from '@docknetwork/crypto-wasm-ts'
const builder = new PresentationBuilder();
builder.addCredential(credential, publicKey)
builder.markAttributesRevealed(0, new Set<string>(['credentialSubject.fname', 'credentialSubject.lname']));
builder.enforceBounds(0, 'credentialSubject.BMI', 10, 40, 'random', snarkProvingKey);

const presentation = builder.finalize();

In [6]:
console.log(presentation)

Presentation {
  _version: '0.9.0',
  spec: PresentationSpecification {
    credentials: [ [Object] ],
    attributeEqualities: [],
    boundedPseudonyms: {},
    unboundedPseudonyms: {}
  },
  proof: CompositeProof {
    value: Uint8Array(828) [
        2,   0,   0,   0,   0,   0,   0,   0,  11, 148, 185, 249,
       27,  97,  34, 240, 106, 156, 186, 251, 148,  40, 127, 237,
      116, 203,  26, 182, 134, 103,   7,  30, 102, 170, 243, 185,
      203, 226,  10, 161, 190, 135,  40, 223,  32,  31,  20,  46,
       29, 111, 213,  56, 111,  17,  72,  13,  46, 130,  17, 189,
       98, 142, 242,  20,  93,  93, 138, 131,  99, 227, 102,  17,
      131,  36, 232, 201, 252, 253,  96, 103,  58,  13,  33,  71,
      201, 172,  39, 245, 172, 227, 142,  36,  43,  70, 127, 251,
      122, 240, 249,   1,
      ... 728 more items
    ]
  },
  attributeCiphertexts: undefined,
  blindedAttributeCiphertexts: undefined,
  context: undefined,
  nonce: undefined
}


## sending data to Verifier
data: 
1. Params to reveal
2. Schema
3. Proof
4. requested range for the range proof

All these info and more are stored in the "presentation object". Note that this object is a completely serializable JSON object.

In [11]:
const receivedObjectOnVerifierSide = presentation.toJSON()
receivedObjectOnVerifierSide  // Assume this was serialized and sent across an API to the verifier.
//presentation.spec.credentials[0]

{
  version: '0.7.0',
  schema: {
    details: '{"jsonSchema":{"$schema":"http://json-schema.org/draft-07/schema#","definitions":{"encryptableCompString":{"type":"string"},"encryptableString":{"type":"string"}},"properties":{"credentialSubject":{"properties":{"BMI":{"minimum":0,"multipleOf":0.01,"type":"number"},"fname":{"type":"string"},"lname":{"type":"string"}},"type":"object"},"proof":{"properties":{"type":{"type":"string"}},"type":"object"}},"type":"object"},"parsingOptions":{"defaultDecimalPlaces":0,"defaultMinimumDate":-17592186044415,"defaultMinimumInteger":-4294967295,"useDefaults":false}}',
    id: 'data:application/json;charset=utf-8,',
    type: 'JsonSchemaValidator2018',
    version: '0.5.0'
  },
  revealedAttributes: { credentialSubject: { fname: 'John', lname: 'Smith' } },
  bounds: { credentialSubject: { BMI: [Array] } },
  sigType: 'Bls12381BBSSignatureDock2023'
}


In [8]:
import { Presentation } from '@docknetwork/crypto-wasm-ts'

const predicateParams = new Map([['random', snarkVerifyingKey]]);

const recreatedPres = Presentation.fromJSON(receivedObjectOnVerifierSide)
recreatedPres.verify([publicKey], undefined, predicateParams, undefined, undefined, undefined)

{ verified: true, error: undefined }
